In [33]:
%config IPCompleter.greedy=True
import numpy as np
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import time
import re
import scipy
from datetime import time, timedelta
import seaborn as sns
import datetime as dt
#pd.set_option('display.max_columns', 200)

import warnings
warnings.filterwarnings("ignore")

In [34]:
'''
*****自定义数据提取的代码*****
输入：
    sql:常规sql代码

输出：
    dataframe格式数据表格
    
备注：
    当sql代码出错的时候，函数会返回sql的报错内容
'''
def get_data_MIMIC(sql):
    conn = psycopg2.connect(database="mimiciv2_2",user="postgres",
                           password="postgres",
                            host="10.245.99.98",
                            #host="172.16.2.76",
                            port="5433")
    cur = conn.cursor()
    try:
        cur.execute(sql)
        #获取表的所有字段名称
        coloumns = [row[0] for row in cur.description]
        result = [[str(item) for item in row] for row in cur.fetchall()]
        return pd.DataFrame(result,columns=coloumns)
    except Exception as ex:
        print(ex)
    finally:
        conn.close()

In [35]:
def trans_id(df):
    for i in ['subject_id','hadm_id','stay_id']:
        if i in df.columns.tolist():
            if df[i].dtype == object:
                if ((df[i]== 'None')|(df[i]== 'Null')).any():
                    df[i].replace('None',np.nan,inplace = True)
                    df[i].replace('Null',np.nan,inplace = True)
                    df[i] = df[i].astype(float)
                else:
                    df[i] = df[i].astype(float)

In [36]:
def trans_time(t):
    if t == 'None':
        return np.datetime64('NaT')
    elif pd.isna(t):
        return np.datetime64('NaT')
    else:
        return np.datetime64(t)

In [37]:
def trans_float(df):
    for i in df.columns:
        if df[i].dtype == object:
            if ((df[i]== 'None')|(df[i]== 'Null')).any():
                df[i].replace('None',np.nan,inplace = True)
                df[i].replace('Null',np.nan,inplace = True)
                df[i] = df[i].astype(float)
            else:
                df[i] = df[i].astype(float)

In [38]:
#mdro = pd.read_csv('MDRO_AFTERIMP_Rmice.csv')

In [39]:
dev = pd.read_csv('../1.MDRO_Predict_Models/dev_df_revise.csv')
val = pd.read_csv('../1.MDRO_Predict_Models/val_df_revise.csv')

In [40]:
mdro = pd.concat([dev,val])

In [41]:
mdro

,stay_id,stay_chart_included_mdr_neg_tot,stay_chart_included_mdr_pos_tot,actual_year_group,gender,admission_age,race,weight,hospstay_seq,icustay_rank,...,sapsii,oasis,los_icu,los_hospital,hospital_expire_flag,stay_chart_included_mdr,RPR,NLR,shock_index,previous_mdr_90d
0,31205490,0,0,2011 - 2013,1,46.275517,WHITE,72.2,1,1,...,23,18,1.34,2.994444,0,0,0.035758,21.976744,0.753425,0
1,33685454,0,0,2009 - 2011,1,81.592179,WHITE,53.0,1,1,...,31,29,6.18,14.172917,0,0,0.071359,9.885057,0.591304,0
2,31090461,0,0,2010 - 2012,1,82.729467,WHITE,48.0,2,2,...,37,26,3.89,5.872222,0,0,0.061597,42.000000,0.796610,0
3,34807493,0,0,2012 - 2014,1,81.330235,WHITE,55.0,2,3,...,44,36,2.02,2.834028,0,0,0.030719,3.128440,0.772414,0
4,33987268,0,0,2012 - 2014,1,81.280232,WHITE,60.6,1,1,...,46,40,4.98,17.090278,0,0,0.047584,26.000000,1.425287,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8140,33110615,0,0,2017 - 2019,0,71.410627,WHITE,99.3,2,2,...,32,23,1.67,22.018056,0,0,0.061303,5.923729,0.979592,0
8141,35175329,0,0,2017 - 2019,0,89.355006,ASIAN - CHINESE,63.7,2,3,...,51,24,2.38,10.093056,0,0,0.160396,11.895522,0.919192,0
8142,33417212,0,0,2017 - 2019,0,89.177540,ASIAN - CHINESE,68.4,1,1,...,35,16,6.00,8.873611,0,0,0.039073,8.886598,0.652778,0
8143,32880062,0,0,2017 - 2019,0,47.881524,UNKNOWN,70.0,1,1,...,29,33,2.85,10.683333,0,0,0.128846,3.707692,0.914894,0


In [42]:
heart_failure = get_data_MIMIC('''
set search_path to mimiciv;
with hf as (
select * 
from mimiciv_hosp.d_icd_diagnoses
where long_title like '%heart%failure%' and long_title not like '%without heart failure%'
),
hf_id as (
select dicd.hadm_id, 1 as heart_failure
from mimiciv_hosp.diagnoses_icd dicd
right join hf
on dicd.icd_code = hf.icd_code)

select ide.stay_id, heart_failure
from mimiciv_derived.icustay_detail ide
left join hf_id
on ide.hadm_id = hf_id.hadm_id
''')

In [43]:
trans_id(heart_failure)
heart_failure = heart_failure.drop_duplicates()

In [44]:
heart_failure['heart_failure'] =  heart_failure['heart_failure'].apply(lambda x:1 if x== '1' else 0)

In [45]:
mdro['previous_mdr_90d'] = (mdro['previous_mdrpos_lessthan90d'] + mdro['previous_mdrneg_lessthan90d']).apply(lambda x:1 if x>=1 else 0)

In [46]:
mdro['liver_disease'] = (mdro['mild_liver_disease'] + mdro['severe_liver_disease']).apply(lambda x:1 if x >=1 else 0)

mdro['diabetes'] = (mdro['diabetes_without_cc'] + mdro['diabetes_with_cc']).apply(lambda x:1 if x >=1 else 0)

In [47]:
mdro = pd.merge(mdro,heart_failure, how = 'left', on = ['stay_id'])

In [48]:
mdro.columns.to_list()

['stay_id',
 'stay_chart_included_mdr_neg_tot',
 'stay_chart_included_mdr_pos_tot',
 'actual_year_group',
 'gender',
 'admission_age',
 'race',
 'weight',
 'hospstay_seq',
 'icustay_rank',
 'los_hosp_beforeicu',
 'previous_mdrpos_lessthan90d',
 'previous_mdrneg_lessthan90d',
 'last_careunit',
 'nasal_swab',
 'blood_culture',
 'urine_culture',
 'sputum_culture',
 'stool_culture',
 'pan_culture',
 'bal_fluid_culture',
 'wound_culture',
 'rectal_swab',
 'csf_culture',
 'before_adm_90d',
 'carbapenem',
 'ceph_4th',
 'ceph_3th',
 'ceph_1st',
 'tobramycin',
 'linezolid',
 'aztreonam',
 'gentamicin',
 'piperacillin_tazobactam',
 'ciprofloxacin',
 'metronidazole',
 'levofloxacin',
 'vancomycin',
 'sulfameth_trimethoprim',
 'clindamycin',
 'erythromycin',
 'azithromycin',
 'doxycycline',
 'ampicillin',
 'nafcillin',
 'aniongap',
 'bicarbonate',
 'bun',
 'calcium',
 'chloride',
 'creatinine',
 'glucose',
 'sodium',
 'potassium',
 'specimen_id',
 'basophils_abs',
 'eosinophils_abs',
 'lymphocytes

In [56]:
selected_features_col = ['gender',
 'admission_age',
 'race',
'last_careunit',
 'weight',
 'hospstay_seq',
 'icustay_rank',
 'los_hosp_beforeicu',
'previous_mdr_90d',
'admission_location_EMERGENCY.ROOM',
 'admission_location_TRANSFER.FROM.HOSPITAL',
 'admission_location_TRANSFER.FROM.SKILLED.NURSING.FACILITY',
 'admission_location_other',
'history_unintentional_weight_loss_10_lbs',
 'history_assistive_devices_Cane',
 'history_assistive_devices_None',
 'history_assistive_devices_Other',
 'history_assistive_devices_Walker',
 'history_assistive_devices_Wheelchair',
 'history_currently_pain',
 'history_dialysis',
 'history_difficulty_swallowing',
 'history_iv_access',
 'history_past_Anemia',
 'history_past_Asthma',
 'history_past_COPD',
 'history_past_Diabetes...Insulin',
 'history_past_Diabetes...Oral.Agent',
 'history_past_ETOH',
 'history_past_GI.Bleed',
 'history_past_HEMO.or.PD',
 'history_past_Hepatitis',
 'history_past_Liver.Failure',
 'history_past_Pancreatitis',
 'history_past_Renal.Failure',
 'history_past_Seizures',
 'history_past_Smoker',
 'history_cv_Angina',
 'history_cv_Arrhythmias',
 'history_cv_CAD',
 'history_cv_CHF',
 'history_cv_CVA',
 'history_cv_Hypertension',
 'history_cv_MI',
 'history_cv_PVD',
 'history_cv_Pacemaker',
'calcium', 'aniongap', 'potassium', 'bicarbonate',
 'glucose', 'bun', 'sodium', 'chloride', 'creatinine',

'eosinophils', 'monocytes_abs',
 'neutrophils_abs', 'basophils', 'monocytes', 'neutrophils',
 'eosinophils_abs', 'basophils_abs',
 'lymphocytes', 'lymphocytes_abs', 'ptt', 'pt',
 'inr', 'hematocrit', 'hemoglobin',
 'mch', 'mchc', 'mcv', 'platelet', 'rbc', 'rdw',
 'wbc', 'heart_rate', 'spo2', 'sbp', 'dbp', 'mbp', 'resp_rate', 'temperature','RPR',
 'NLR', 'mg','phosphate',
'liver_disease',
 'diabetes',
 'heart_failure',                         
#  'admission_location_EMERGENCY.ROOM',
#  'admission_location_TRANSFER.FROM.HOSPITAL',
#  'admission_location_TRANSFER.FROM.SKILLED.NURSING.FACILITY',
'cerebrovascular_disease',
 'congestive_heart_failure',
 'chronic_pulmonary_disease',
 'diabetes_with_cc',
 'renal_disease',
 'malignant_cancer',
 'severe_liver_disease',
'charlson_comorbidity_index',
 'immu_duration',
 'ant_duration',
'ant_1stday',
 'immu_1stday',
 'vaso_beforeicu',
 'vaso_1stday',
 'surgical',
 'invasive_line_1stday',
 'ventilation_1stday',
 'ent_nut',
 'par_nut',
 'gcs',
'sofa',
'apsiii',
'shock_index',
'los_icu',
'los_hospital',
'hospital_expire_flag',                       
'stay_chart_included_mdr'
]

In [50]:
from tableone import TableOne

In [51]:
NAME_RE_DICT = {'sbp':'SBP',
 'weight':'weight',
 'heart_rate':'HR',
 'lactate':'lactate',
 'neutrophils_abs':'absolute neutrophil count',
 'anttestR_any':'previous any bacterial resistance',
 'PIPERACILLIN.TAZO_anttest_beforeicu90d':'previous Pip-taz resistance',
 'TOBRAMYCIN_anttest_beforeicu90d':'previous Tobramycin resistance',
 'CEFEPIME_anttest_beforeicu90d':'previous Cefepime resistance',
 'history_past_COPD':'history of COPD',
 'history_iv_access':'history of IV access',
 'immu_duration':'days of immunosuppressant use before ICU',
 'bun':'BUN',
 'chloride':'chloride',
 'spo2':'SpO2',
 'eosinophils_abs':'absolute eosinophil count',
 'age':'age',
 'before_adm_90d':'previous admission within 90d',
 'charlson_comorbidity_index':'charlson comorbidity index',
 'ant_duration':'days of antimicrobial use before ICU',
 'los_hosp_beforeicu':'days of hospital stay before ICU',
 'chronic_pulmonary_disease':'chronic pulmonary disease',
 'creatinine':'creatinine',
 'hemoglobin':'hemoglobin',
 'inr':'INR',
 'icustay_rank':'No. of ICU admissions',
 'CEFTRIAXONE_anttest_beforeicu90d':'previous Ceftriaxone resistance',
 'bicarbonate':'bicarbonate',
 'CIPROFLOXACIN_anttest_beforeicu90d':'previous Ciprofloxacin resistance',
 'hospstay_seq':'No. of hospital admissions',
 'history_assistive_devices_Wheelchair':'Wheelchair used as assistive devices',
 'gcs':'GCS',
 'CEFTAZIDIME_anttest_beforeicu90d':'previous Ceftazidime resistance',
 'dbp':'DBP',
 'gender':'gender.Male',
 'sodium':'sodium',
 'temperature':'temperature',
 'basophils_abs':'absolute basophil count',
 'mch':'MCH',
 'mchc':'MCHC',
 'lymphocytes_abs':'absolute lymphocyte count',
 'glucose':'glucose',
 'wbc':'WBC',
 'platelet':'platelet',
 'resp_rate':'RR',
 'GENTAMICIN_anttest_beforeicu90d':'previous Gentamicin resistance',
 'calcium':'calcium',
 'MEROPENEM_anttest_beforeicu90d':'previous Meropenem resistance',
 'history_difficulty_swallowing':'history of difficulty swallowing',
 'mcv':'MCV',
 'history_cv_Pacemaker':'history of CV Pacemaker',
 'potassium':'potassium',
 'TRIMETHOPRIM.SULFA_anttest_beforeicu90d':'previous TMP-SMX resistance',
 'shock_index':'shock index',
 'aniongap':'aniongap',
 'rdw':'RDW',
 'sofa':'SOFA',
 'apsiii':'APSIII',
  'mg':'serum magnesium',
  'phosphate': 'serum phosphate'}

In [52]:
# Function to map each subcategory to a main category in English
def map_to_main_category_english(race):
    if 'ASIAN' in race:
        return 'Asian'
    elif race in ['BLACK/AFRICAN AMERICAN', 'BLACK/CAPE VERDEAN', 'BLACK/CARIBBEAN ISLAND', 'BLACK/AFRICAN']:
        return 'Black/African American'
    elif 'HISPANIC' in race or 'LATINO' in race or race == 'PORTUGUESE':
        return 'Hispanic/Latino'
    elif 'WHITE' in race:
        return 'White'
    else:
        return 'Other'

# Apply the function to the race column
mdro['race'] = mdro['race'].apply(map_to_main_category_english)


In [53]:
mdro['race'].value_counts()

White                     21270
Other                      4521
Black/African American     3542
Hispanic/Latino            1335
Asian                       983
Name: race, dtype: int64

In [54]:
mdro['gender'].value_counts()

0    17631
1    14020
Name: gender, dtype: int64

In [57]:
columns_com = selected_features_col

nonnormal_com = [
 'admission_age', 'weight', 'hospstay_seq',
 'icustay_rank', 'los_hosp_beforeicu',
'calcium', 'aniongap', 'potassium', 'bicarbonate',
 'glucose', 'bun', 'sodium', 'chloride', 'creatinine',

'eosinophils', 'monocytes_abs',
 'neutrophils_abs', 'basophils', 'monocytes', 'neutrophils',
 'eosinophils_abs', 'basophils_abs',
 'lymphocytes', 'lymphocytes_abs', 'ptt', 'pt',
 'inr', 'hematocrit', 'hemoglobin',
 'mch', 'mchc', 'mcv', 'platelet', 'rbc', 'rdw',
 'wbc', 'heart_rate', 'spo2',
 'sbp', 'dbp', 'mbp', 'resp_rate', 'temperature','RPR',
 'NLR',
'charlson_comorbidity_index',
 'immu_duration', 'ant_duration','los_icu',
'los_hospital',
 'gcs','sofa','apsiii','shock_index', 'mg','phosphate'
]
category_com = [
'gender', 'race','last_careunit', 'previous_mdr_90d','admission_location_EMERGENCY.ROOM',
 'admission_location_TRANSFER.FROM.HOSPITAL',
 'admission_location_TRANSFER.FROM.SKILLED.NURSING.FACILITY',
 'admission_location_other',
'history_unintentional_weight_loss_10_lbs',
 'history_assistive_devices_Cane',
 'history_assistive_devices_None',
 'history_assistive_devices_Other',
 'history_assistive_devices_Walker',
 'history_assistive_devices_Wheelchair',
 'history_currently_pain',
 'history_dialysis',
 'history_difficulty_swallowing',
 'history_iv_access',
 'history_past_Anemia',
 'history_past_Asthma',
 'history_past_COPD',
 'history_past_Diabetes...Insulin',
 'history_past_Diabetes...Oral.Agent',
 'history_past_ETOH',
 'history_past_GI.Bleed',
 'history_past_HEMO.or.PD',
 'history_past_Hepatitis',
 'history_past_Liver.Failure',
 'history_past_Pancreatitis',
 'history_past_Renal.Failure',
 'history_past_Seizures',
 'history_past_Smoker',
 'history_cv_Angina',
 'history_cv_Arrhythmias',
 'history_cv_CAD',
 'history_cv_CHF',
 'history_cv_CVA',
 'history_cv_Hypertension',
 'history_cv_MI',
 'history_cv_PVD',
 'history_cv_Pacemaker', 'liver_disease',
 'diabetes', 'heart_failure', 'cerebrovascular_disease', 'congestive_heart_failure', 'chronic_pulmonary_disease', 'diabetes_with_cc', 'renal_disease',
 'malignant_cancer', 'severe_liver_disease', 
 'ant_1stday', 'immu_1stday', 'vaso_beforeicu', 'vaso_1stday',
 'surgical', 'invasive_line_1stday', 'ventilation_1stday', 'ent_nut', 'par_nut','hospital_expire_flag'
]
groupby = ['stay_chart_included_mdr']
tableone_com = TableOne(mdro,
                        nonnormal=nonnormal_com,
                        categorical=category_com,
                        columns=columns_com,
                        rename = NAME_RE_DICT,
                        groupby=groupby,
                        label_suffix=True,
                        pval=True,normal_test = True,tukey_test = True,dip_test = True,
                        htest_name=True)

In [58]:
tableone_com.to_excel('../1.MDRO_Predict_Models/results/MIMICIV_MDRO_tableone_revise2.xlsx')